<a href="https://colab.research.google.com/github/Brettem/My_Python_Stuff/blob/master/Linear_G_code_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import sympy as sp

In [2]:
# Critical Inputs

Trial = "TEMPLATE"

V_star = 0.2

H_star = 8.0


print(f"1/H* = {1/H_star}")
if V_star <= 1/H_star:
  print("Will accumulate")
else:
  print("Will not accumulate")

1/H* = 0.125
Will not accumulate


In [3]:
# Hardcode Inputs

D_f = 1.75 # Diameter of filament in 
D_n = 0.4 # Diameter of Nozzle

T_n = 240 # Nozzle Temp (°C)
T_b = 100 # Bed Temp (°C)

X0 = 50 # Starting move x-coordinate (mm)
Y0 = 105 # Starting move y-coordinate (mm)

# Z0 = 5 # Height of first layer, [AKA: H] (mm)

Zα = 150 # Height of α test 

F1 = 300 # Second coordinates F-speed, [move speed of nozzle to second coordinates] (mm/min)
# X1 = 200 # Second move x-coordinate (mm)
# E1 = 100 # Second move E-amount, [amount of filament fed into the extruder during move to second coordinates] (mm)
S = 0.66 # Flow Multiplyer, [M221]

L1 = 150 # Length of move (mm)

α = 1.63 # Die-Swell Constant [Prediction]
a = α

In [4]:
# Formulaic Inputs

L = L1

H = H_star*α*D_n

# E = (L/(S*V_star))*(((α*D_n)/D_f)**2) # E as manipulated variable
E = L # only use when NOT using E as manipulated variable

S = (L/(E*V_star))*(((α*D_n)/D_f)**2)

Z0 = H
E1 = E

X1 = X0 + L1

print(f"H = {H}")
print(f"E = {E}")
print(f"S = {S}")
print(f"X1 = {X1}")

H = 5.216
E = 150
S = 0.6940473469387756
X1 = 200


In [ ]:
# Functions

class gcode_generator:
    def __init__(self, move, layer, X_0 = 100, Y_0 = 80, X_wipe, Y_wipe, X, Y, Z, E, F):
        self.move = move
        self.layer = layer
        self.X_0 = X_0
        self.Y_0 = Y_0
        self.X_wipe = X_wipe
        self.Y_wipe = Y_wipe
        self.X = X
        self.Y = Y
        self.Z = Z
        self.E = E
        self.F = F
        
    def wipe(self):
        return X = self.X_0 - 50
    
    def X_move(self):
        if move == 0:
            X = X_0
        elif move > 0 and X != X_1:
            X = X_1
        else:
            X = X_0
        return X
    
    def Y_move(self):
        if move == 0:
            Y = Y_0
        elif move > 0 and Y != Y_1:
            Y = Y_1
        else:
            Y = Y_0
        return Y

In [7]:
# Writing of G-code

D1 = [f'M73 P0 R0 \n',
      f'M201 X9000 Y9000 Z500 E10000 ; sets maximum accelerations, mm/sec^2 \n',
      f'M203 X500 Y500 Z12 E120 ; sets maximum feedrates, mm/sec \n',
      f'M204 P2000 R1500 T2000 ; sets acceleration (P, T) and retract acceleration (R), mm/sec^2 \n',
      f'M205 X10.00 Y10.00 Z0.20 E4.50 ; sets the jerk limits, mm/sec \n',
      f'M205 S0 T0 ; sets the minimum extruding and travel feed rate, mm/sec \n',
      f'M107 ; turns off fan \n',
      f'M862.3 P "MK2.5S" ; printer model check \n',
      f'M862.1 P0.4 ; nozzle diameter check \n',
      f'G90 ; use absolute coordinates \n',
      f'M83  ; extruder relative mode \n',
      
      f'\n'
      
      f'M104 S{T_n} ; set extruder temp \n',
      f'M140 S{T_b} ; set bed temp \n',
      f'M190 S{T_b} ; wait for bed temp \n',
      f'M109 S{T_n} ; wait for extruder temp \n',

      f'\n',
      
      f'G28 W ; home all without mesh bed level \n',
      f'G80 ; mesh bed leveling \n',
      f'G1 Y-3.0 F1000.0 ; go outside print area \n',
      f'G92 E0.0 \n',
      f'G1 X60.0 E9.0 F1000.0 ; intro line \n',
      f'M73 P14 R0 \n',
      f'G1 X100.0 E12.5 F1000.0 ; intro line \n',
      f'G92 E0.0 \n',
      f'G21 ; set units to millimeters \n',
      f'G90 ; use absolute coordinates \n',
      f'M83 ; use relative distances for extrusion \n',
      f'M900 K0.05 ; Filament gcode LA 1.5 \n',
      f'M900 K30 ; Filament gcode LA 1.0 \n',
      f'G92 E0.0 \n',

      f'M221 S{S} \n',
      
      f'G1 Z0.200 F10800.000 \n',
     
      f'\n',
      f'\n']

M1 = [f'G1 X{X0} Y{Y0} Z{Z0} \n',
      f'M204 S1000 \n',
      f'G1 X{X1} F{F1} E{E1} \n']

M2 = [f'G1 X{X0} Y{Y0} Z{Zα} \n',
      f'M204 S1000 \n',
      f'G1 X{X1} F{F1} E{E1} \n']

D2 = [f'\n',
      f'\n',
      
      f'G1 F10800.000 \n',
      f'G4 ; wait \n',
      f'M104 S0 ; turn off temperature \n',
      f'M140 S0 ; turn off heatbed \n',
      f'M107 ; turn off fan \n',
      f'G1 Z30.2 ; Move print head up \n',
      f'M73 P91 R0 \n',
      f'G1 X0 Y200 F3000 ; home X axis \n',
      f'M900 K0 ; reset LA \n',
      f'M84 ; disable motors \n',
      f'M73 P100 R0 \n']
  
# \n is placed to indicate EOL (End of Line) 

with open (f'{Trial}.gcode', 'w') as f:
  f.writelines(D1)
  f.writelines(M1)
  f.writelines(D2)

with open (f'{Trial}_αTest.gcode', 'w') as f:
  f.writelines(D1)
  f.writelines(M2)
  f.writelines(D2)